In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')

### Explore and preprocess data

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.shape

(891, 12)

In [5]:
df.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

In [6]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
df.set_index('PassengerId',inplace=True)
df.drop(['Name','Ticket','Cabin'],axis=1,inplace=True)
df['Sex'].replace('male',1,inplace=True)
df['Sex'].replace('female',0,inplace=True)
df.dropna(inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,1,22.0,1,0,7.2500,S
2,1,1,0,38.0,1,0,71.2833,C
3,1,3,0,26.0,0,0,7.9250,S
4,1,1,0,35.0,1,0,53.1000,S
5,0,3,1,35.0,0,0,8.0500,S


In [8]:
df.shape

(712, 8)

In [9]:
X = df.drop('Survived',axis=1)
y = df['Survived']

In [10]:
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,1,22.0,1,0,7.2500,S
2,1,0,38.0,1,0,71.2833,C
3,3,0,26.0,0,0,7.9250,S
4,1,0,35.0,1,0,53.1000,S
5,3,1,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,3,0,39.0,0,5,29.1250,Q
887,2,1,27.0,0,0,13.0000,S
888,1,0,19.0,0,0,30.0000,S


In [11]:
y

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
886    0
887    0
888    1
890    1
891    0
Name: Survived, Length: 712, dtype: int64

#### convert embarkation port to dummy variables, first using pd.get_dummies then OneHotEncoder

In [12]:
X = pd.get_dummies(data=X,prefix='emb',drop_first=True)

In [13]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,emb_Q,emb_S
PassengerId,,,,,,,,
1,3,1,22.0,1,0,7.2500,0,1
2,1,0,38.0,1,0,71.2833,0,0
3,3,0,26.0,0,0,7.9250,0,1
4,1,0,35.0,1,0,53.1000,0,1
5,3,1,35.0,0,0,8.0500,0,1


In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=123)

### apply our first model - XGBoost

In [15]:
import xgboost as xgb

xg_cl = xgb.XGBClassifier(seed=123,verbosity = 0)

In [16]:
xg_cl.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=123,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=0)

In [17]:
y_pred = xg_cl.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred,y_test))

0.7972027972027972


In [19]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth': [2,5,10],
    'n_estimators': [50,100,300],
    'learning_rate': [0.1, 0.01, 0.05]
}

base_est = xgb.XGBClassifier(seed=123,verbosity = 0)

grid_cv = GridSearchCV(
    estimator=base_est,
    param_grid=params,
    scoring = 'accuracy',
    cv=5
)

In [20]:
grid_cv.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight

In [21]:
grid_cv.best_score_

0.8313771153547587

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

In [24]:
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y,test_size=.2,random_state=123)

In [25]:
base_est = xgb.XGBClassifier(seed=123,verbosity = 0)

grid_cv = GridSearchCV(
    estimator=base_est,
    param_grid=params,
    scoring = 'accuracy',
    cv=5
)

grid_cv.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight

In [26]:
grid_cv.best_score_

0.8296227293898465